In [1]:
import pandas as pd
import numpy as np
import os
import duckdb
from pathlib import Path
from tqdm import tqdm
from scipy.sparse import csr_matrix, hstack
from scipy.sparse.linalg import lsqr
import warnings
import plotly.graph_objects as go
import plotly.express as px
import scipy.sparse as sp
from scipy.sparse.linalg import lsqr
from scipy.stats import chi2
import scipy.stats as stats
import calendar


# --- Setup ---
warnings.filterwarnings("ignore")
DATA_FOLDER = Path("/home/akimovh/predatory_pricing_airlines/data")
PLOT_FOLDER = Path("/home/akimovh/predatory_pricing_airlines/plots")
con = duckdb.connect(str(DATA_FOLDER / "duckdb" / "airlines.duckdb"))
con.execute(f"PRAGMA threads={os.cpu_count()}")
# Create color palette
color_palette = ['rgba(0, 74, 174, 1)', 'rgba(231, 17, 94, 1)', 'rgba(255, 195, 27, 1)',  'rgba(130, 54, 199, 1)',
                 'rgba(1, 137, 67, 1)', 'rgba(252, 147, 19, 1)', 'rgba(159, 209, 255, 1)', 'rgba(119, 7, 71, 1)']

# HA revenue plot

In [ ]:
costs = con.execute("""
           select t1.*,
                  make_date(t1.Year::INT, (t1.Quarter::INT - 1) * 3 + 1, 1) AS date,
            from T_F41 as t1
            where UNIQUE_CARRIER == 'HA'
                  and REGION == 'D'
            order by date
           """
          ).df()

In [ ]:
# -----------------------
# Prepare data
# -----------------------
rev_plot_data = costs[['date', 'INCOME_PRE_TAX']].copy()
rev_plot_data['INCOME_PRE_TAX'] *= 1000

year_tick_vals = (
    rev_plot_data
    .groupby(rev_plot_data['date'].dt.year)['date']
    .first()
    .to_numpy()
)

unique_years = rev_plot_data['date'].dt.year.unique()

# -----------------------
# Create figure
# -----------------------
fig = go.Figure()

# Income line with markers
fig.add_trace(
    go.Scatter(
        x=rev_plot_data['date'],
        y=rev_plot_data['INCOME_PRE_TAX'],
        mode='lines+markers',
        line=dict(
            color='rgba(0, 74, 174, 1)',
            width=3
        ),
        marker=dict(
            color='rgba(0, 74, 174, 1)',
            size=10,
            symbol='circle'
        )
    )
)

# Horizontal zero line
fig.add_shape(
    type='line',
    x0=rev_plot_data['date'].min(), y0=0,
    x1=rev_plot_data['date'].max(), y1=0,
    line=dict(color='black', width=1)
)

# Southwest entry vertical line
fig.add_shape(
    type='line',
    x0=pd.to_datetime('2019-04-01'), y0=rev_plot_data['INCOME_PRE_TAX'].min(),
    x1=pd.to_datetime('2019-04-01'), y1=rev_plot_data['INCOME_PRE_TAX'].max(),
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash='dash')
)

fig.add_annotation(
    x=pd.to_datetime('2019-04-01'),
    y=rev_plot_data['INCOME_PRE_TAX'].max() + 10_000_000,
    text='SW entry',
    showarrow=False,
    font=dict(
        family='Arial',
        size=20,
        color='rgba(231, 17, 94, 1)'
    )
)

# COVID-19 shaded period
fig.add_shape(
    type='rect',
    x0='2020-02-01',
    x1='2021-04-01',
    y0=-200_000_000,
    y1=180_000_000,
    fillcolor='lightgray',
    opacity=0.3,
    layer='below',
    line_width=0
)

fig.add_annotation(
    x='2020-08-10',
    y=130_000_000,
    text='COVID-19',
    showarrow=False,
    align='center',
    font=dict(
        family='Arial',
        size=20,
        color='rgba(231, 17, 94, 1)'
    ),
    bgcolor='white',
    opacity=0.7,
    borderpad=4
)

# -----------------------
# Layout
# -----------------------
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family='Arial',
        size=20
    ),
    xaxis=dict(
        title='Date (Quarter)',
        title_font=dict(size=22),
        tickmode='array',
        tickvals=year_tick_vals,
        ticktext=unique_years,
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title='Pre-tax Net Income (Millions US Dollars)',
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True,
        tickformat=','
    ),
    margin=dict(t=50, b=80, l=100, r=40),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)'
)

fig.show()

fig.write_image(PLOT_FOLDER / "HA_income.png",scale=3)


# Price-cost analysis

### Price

In [ ]:
inner_HI = con.execute("""select t1.*,
                                 case when TkCarrier = 'HA' then 'Hawaiian Airlines'
                                      when TkCarrier = 'WN' then 'Southwest Airlines'
                                      else 'Aloha Airlines' end as TkCarrier_name
                       from DB1B_processed t1
                       where OriginState='HI' AND DestState='HI' and TkCarrier in ['HA', 'WN', 'AQ'] """).df()

In [ ]:
fig = go.Figure()

# Data setup
price_plot_data = inner_HI.groupby(['date', "TkCarrier_name"], as_index=False)[['Passengers', 'sum_fares']].sum()
price_plot_data = price_plot_data.query("Passengers > 200")
price_plot_data['price'] = price_plot_data.sum_fares / price_plot_data.Passengers
price_plot_data = price_plot_data.query("date > '2016-01-01'")

fig1 = px.line(price_plot_data, x="date", y="price", color='TkCarrier_name', symbol='TkCarrier_name')

# Re-apply custom styling to traces
for i, trace in enumerate(fig1.data):
    trace.line.color = color_palette[i]
    trace.marker.color = color_palette[i]
    trace.marker.size = 10
    trace.legendgroup = trace.name
    trace.showlegend = True
    fig.add_trace(trace)

# COVID rectangle
fig.add_shape(
    type="rect",
    x0="2020-02-01",
    x1="2021-04-01",
    y0=30,
    y1=110,
    fillcolor="lightgray",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# COVID annotation
fig.add_annotation(
    x='2020-09-01',
    y=100,
    text="COVID-19",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    ),
    align="center",
    bgcolor="white",
    opacity=0.7,
    borderpad=4,
)

# SW entry line
fig.add_shape(
    type="line",
    x0=pd.to_datetime("2019-04-01"), y0=30,
    x1=pd.to_datetime("2019-04-01"), y1=110,
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash="dash")
)

# SW entry annotation
fig.add_annotation(
    x=pd.to_datetime("2019-04-01"),
    y=112,
    text="SW entry",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    )
)

# Merger line
fig.add_shape(
    type="line",
    x0=pd.to_datetime("2023-10-01"), y0=30,
    x1=pd.to_datetime("2023-10-01"), y1=110,
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash="dash")
)

# Merger annotation
fig.add_annotation(
    x=pd.to_datetime("2023-10-01"),
    y=112,
    text="AA & HA merger announcment",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    )
)

# Layout update for presentation
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family="Arial",
        size=20
    ),
    xaxis=dict(
        title="Date (Quarter)",
        title_font=dict(size=22),
        tickmode='array',
        tickvals=year_tick_vals,
        ticktext=unique_years,
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title="Average Fare (US dollars)",
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    legend=dict(
        font=dict(size=18),
        orientation="h",
        y=-0.15,
        x=0.5,
        xanchor="center",
        bordercolor="black",
        borderwidth=1
    ),
    margin=dict(t=50, b=100, l=100, r=40)
)

fig.show()

# High-res export for LaTeX presentation
fig.write_image( PLOT_FOLDER/"HI_II_avg_fares.png", scale=3 )


### Costs

In [ ]:
cost_data = con.execute("""
WITH 
-- Financials (System-wide Costs)
financials AS (
    SELECT UNIQUE_CARRIER, YEAR, QUARTER,
           (FLYING_OPS + MAINTENANCE + AIRCFT_SERVICES + TRANS_EXPENSES + PAX_SERVICE + PROMOTION_SALES) AS dist_cost,
           PAX_SERVICE AS pas_cost
    FROM T_F41
    WHERE REGION = 'D' AND UNIQUE_CARRIER IN ('WN', 'HA', 'AQ')
),

-- System-Wide Stats (The Denominator)
system_stats AS (
    SELECT UNIQUE_CARRIER, YEAR, QUARTER,
           SUM(SEATS * DISTANCE) AS system_asm,
           SUM(DEPARTURES_PERFORMED * DISTANCE) / NULLIF(SUM(DEPARTURES_PERFORMED), 0) AS system_avg_dist
    FROM T100
    WHERE DEPARTURES_PERFORMED > 0 AND CLASS IN ('F','E','C')
    GROUP BY 1, 2, 3
),

-- Target Route Stats (The Numerator)
route_stats AS (
    SELECT t1.UNIQUE_CARRIER, t1.YEAR, t1.QUARTER, t2.market_id,
           SUM(t1.PASSENGERS) / NULLIF(SUM(t1.SEATS), 0) AS load_factor,
           SUM(t1.DEPARTURES_PERFORMED * t1.DISTANCE) / NULLIF(SUM(t1.DEPARTURES_PERFORMED), 0) AS segment_dist
    FROM T100 t1
    JOIN routes t2 ON t1.ORIGIN_CITY_MARKET_ID = t2.ORIGIN_CITY_MARKET_ID 
                  AND t1.DEST_CITY_MARKET_ID = t2.DEST_CITY_MARKET_ID
    WHERE t2.market_id IN (SELECT DISTINCT market_id FROM final_panel WHERE target_routes = 1)
      AND t1.DEPARTURES_PERFORMED > 0 AND t1.CLASS IN ('F','E','C')
    GROUP BY 1, 2, 3, 4
)

-- 4. Final Calculation
SELECT 
    r.*,
    case when UNIQUE_CARRIER = 'HA' then 'Hawaiian Airlines'
         when UNIQUE_CARRIER = 'WN' then 'Southwest Airlines'
         else 'Aloha Airlines' end as TkCarrier_name, 
    make_date(r.YEAR::INT, (r.QUARTER::INT - 1) * 3 + 1, 1) AS date,
    (
        ((f.dist_cost / s.system_asm) * 1000)      -- Base CASM (cents)
        * r.segment_dist                           -- Segment Distance
        * POW(s.system_avg_dist / r.segment_dist, 0.15) -- scale adjustment
    ) / r.load_factor AS avg_cost

FROM route_stats r
JOIN financials f USING (UNIQUE_CARRIER, YEAR, QUARTER)
JOIN system_stats s USING (UNIQUE_CARRIER, YEAR, QUARTER)
where YEAR >= 2016
ORDER BY r.market_id, r.YEAR, r.QUARTER
""").df()

In [ ]:
fig = go.Figure()

cost_plot_data = cost_data.groupby(['TkCarrier_name', 'date'], as_index = False).avg_cost.mean()

fig1 = px.line(cost_plot_data, x="date", y="avg_cost", color='TkCarrier_name', symbol='TkCarrier_name')

# Re-apply custom styling to traces
for i, trace in enumerate(fig1.data):
    trace.line.color = color_palette[i]
    trace.marker.color = color_palette[i]
    trace.marker.size = 10
    trace.legendgroup = trace.name
    trace.showlegend = True
    fig.add_trace(trace)

# COVID shaded region
fig.add_shape(
    type="rect",
    x0="2020-02-01",
    x1="2021-04-01",
    y0=-20,
    y1=110,
    fillcolor="lightgray",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# COVID annotation
fig.add_annotation(
    x='2020-09-01',
    y=80,
    text="COVID-19",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    ),
    align="center",
    bgcolor="white",
    opacity=0.7,
    borderpad=4,
)

# Southwest entry vertical line
fig.add_shape(
    type="line",
    x0=pd.to_datetime("2019-04-01"), y0=-1000,
    x1=pd.to_datetime("2019-04-01"), y1=100,
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash="dash")
)

# Southwest entry annotation
fig.add_annotation(
    x=pd.to_datetime("2019-04-01"),
    y=102,
    text="SW entry",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color='rgba(231, 17, 94, 1)'
    )
)

# Layout
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family="Arial",
        size=20
    ),
    xaxis=dict(
        title="Date (Quarter)",
        title_font=dict(size=22),
        tickmode='array',
        tickvals=year_tick_vals,
        ticktext=unique_years,
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title="Average Cost per Passenger (US dollars)",
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True,
        range=[0, 105]
    ),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    legend=dict(
        font=dict(size=18),
        orientation="h",
        y=-0.15,
        x=0.5,
        xanchor="center",
        bordercolor="black",
        borderwidth=1
    ),
    margin=dict(t=50, b=100, l=100, r=40)
)

fig.show()

fig.write_image( PLOT_FOLDER/"HI_II_avg_cost.png", scale=3 )

In [ ]:
fig = go.Figure()

markup_plot_data = pd.merge(price_plot_data, cost_plot_data, on = ['date', 'TkCarrier_name'])
markup_plot_data["avg_markup"] = markup_plot_data.price - markup_plot_data.avg_cost 
fig1 = px.line(markup_plot_data, x="date", y="avg_markup", color='TkCarrier_name', symbol='TkCarrier_name')

# Re-apply custom styling to traces
for i, trace in enumerate(fig1.data):
    trace.line.color = color_palette[i]
    trace.marker.color = color_palette[i]
    trace.marker.size = 10
    trace.legendgroup = trace.name
    trace.showlegend = True
    fig.add_trace(trace)

# COVID rectangle
fig.add_shape(
    type="rect",
    x0="2020-02-01",
    x1="2021-04-01",
    y0=-55,
    y1=110,
    fillcolor="lightgray",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# COVID annotation
fig.add_annotation(
    x='2020-09-01',
    y=80,
    text="COVID-19",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    ),
    align="center",
    bgcolor="white",
    opacity=0.7,
    borderpad=4,
)

# Horizontal line at y = 0
fig.add_shape(
    type="line",
    x0=markup_plot_data['date'].min(),
    x1=markup_plot_data['date'].max(),
    y0=0,
    y1=0,
    line=dict(
        color="black",
        width=2,
        dash="solid"
    )
)

# Southwest entry line
fig.add_shape(
    type="line",
    x0=pd.to_datetime("2019-04-01"),
    x1=pd.to_datetime("2019-04-01"),
    y0=-1000,
    y1=100,
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash="dash")
)

# Southwest entry annotation
fig.add_annotation(
    x=pd.to_datetime("2019-04-01"),
    y=102,
    text="SW entry",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color='rgba(231, 17, 94, 1)'
    )
)

# Layout adjustments
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family="Arial",
        size=20
    ),
    xaxis=dict(
        title="Date (Quarter)",
        title_font=dict(size=22),
        tickmode='array',
        tickvals=year_tick_vals,
        ticktext=unique_years,
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title="Average Markup (US dollars)",
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True,
        range=[-50, 105]
    ),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    legend=dict(
        font=dict(size=18),
        orientation="h",
        y=-0.15,
        x=0.5,
        xanchor="center",
        bordercolor="black",
        borderwidth=1
    ),
    margin=dict(t=50, b=100, l=100, r=40)
)

fig.show()

# Export high-resolution image
fig.write_image( PLOT_FOLDER/"HI_II_avg_markup.png", scale=3 )

# Event study

In [ ]:
# Load Data
df = con.execute("select * from final_panel").df()

y_full = df['mean_log_fare'].values

df['date_cat'] = df['date'].astype('category')
df['market_cat'] = df['market_id'].astype('category')

n_rows = len(df)

# Constant
X_const = sp.csc_matrix(np.ones((n_rows, 1)))

# Time Fixed Effects (One-Hot Encoding)
X_time = sp.coo_matrix(
    (np.ones(n_rows), (np.arange(n_rows), df['date_cat'].cat.codes)),
    shape=(n_rows, len(df['date_cat'].cat.categories))
)

# Market Fixed Effects
X_market = sp.coo_matrix(
    (np.ones(n_rows), (np.arange(n_rows), df['market_cat'].cat.codes)),
    shape=(n_rows, len(df['market_cat'].cat.categories))
)

# COVID State Fixed Effects
# Logic: (Origin==State | Dest==State) & covid==1
# Build a matrix for Origin and Dest, sum them, clip to 1 (OR logic), then apply covid mask.

# Get all unique states from both columns to ensure alignment
all_states = np.unique(np.concatenate([df.OriginState.unique(), df.DestState.unique()]))
state_cat = pd.CategoricalDtype(categories=all_states, ordered=True)

orig_codes = df['OriginState'].astype(state_cat).cat.codes
dest_codes = df['DestState'].astype(state_cat).cat.codes

# Matrix of Origins
X_orig = sp.coo_matrix(
    (np.ones(n_rows), (np.arange(n_rows), orig_codes)),
    shape=(n_rows, len(all_states))
)

# Matrix of Dests
X_dest = sp.coo_matrix(
    (np.ones(n_rows), (np.arange(n_rows), dest_codes)),
    shape=(n_rows, len(all_states))
)

# Combine: (Origin OR Dest)
X_state = X_orig + X_dest
X_state.data = np.where(X_state.data > 0, 1, 0) # Boolean OR logic (handles same-state case)

# Apply COVID mask: Zero out rows where covid != 1
# Use multiply by diagonal matrix for speed
covid_diag = sp.diags(df['covid'].values)
X_covid = covid_diag.dot(X_state)

# Combine all parts horizontally
# Drop first column of Time and Market to avoid multicollinearity (Dropping reference)
# Slicing CSC is efficient for columns
X_time_csc = X_time.tocsc()[:, 1:] 
X_market_csc = X_market.tocsc()[:, 1:]

# Final X Matrix
X_full = sp.hstack([X_const, X_time_csc, X_market_csc, X_covid], format='csr')

train_mask_full = (
    ((df.rel_period < -3) | (df.rel_period == -100)) & 
    ((df.hi_routes == 0) | (df.rel_period < -6))
).values

meta_df = df[['rel_period', 'target_routes', 'monopoly_market']].reset_index(drop=True)

In [ ]:
def estimate_coefficients_optimized(idx_mask, y=y_full, X=X_full, 
                                    train_mask=train_mask_full, meta=meta_df):
    # Slice EVERYTHING using the same indices
    y_boot = y[idx_mask]
    X_boot = X[idx_mask]
    train_boot = train_mask[idx_mask]
    meta_boot = meta.iloc[idx_mask].copy() 

    # Filter for Training
    X_train = X_boot[train_boot]
    y_train = y_boot[train_boot]

    # Solve (Sparse LSQR)
    beta = lsqr(X_train, y_train)[0]

    # Predict
    y_hat = X_boot.dot(beta)
    
    # Calculate Residuals
    meta_boot['dif'] = y_boot - y_hat

    # Aggregate
    res_df = meta_boot.query("rel_period > -17 and rel_period < 17")
    
    base_coefs = res_df.query('target_routes == 0').groupby('rel_period')['dif'].mean()
    monopoly_coefs = res_df.query('target_routes == 0 and monopoly_market == 1').groupby('rel_period')['dif'].mean()
    targeted_coefs = res_df.query('target_routes == 1').groupby('rel_period')['dif'].mean()
    
    return base_coefs, monopoly_coefs, targeted_coefs

In [ ]:
full_sample_mask = np.arange(len(df))
base_coefs_main, monopoly_coefs_main, targeted_coefs_main = estimate_coefficients_optimized(full_sample_mask)

In [ ]:
periods = np.arange(-16, 17) 
n_periods = len(periods)
n_bootstraps = 10  # Start small as requested

# Initialize storage arrays (n_bootstraps x n_periods)
base_bootstrap = np.full((n_bootstraps, n_periods), np.nan)
monopoly_bootstrap = np.full((n_bootstraps, n_periods), np.nan)
targeted_bootstrap = np.full((n_bootstraps, n_periods), np.nan)

market_indices_map = df.groupby('market_id').indices

targeted_markets = df[df.target_routes == 1]['market_id'].unique()
non_targeted_markets = df[df.target_routes == 0]['market_id'].unique()

In [ ]:
# SETUP & PRE-COMPUTATION ---
periods = np.arange(-16, 17) 
n_periods = len(periods)
n_bootstraps = 1000  # Scaling up!

# Initialize storage
base_bootstrap = np.full((n_bootstraps, n_periods), np.nan)
monopoly_bootstrap = np.full((n_bootstraps, n_periods), np.nan)
targeted_bootstrap = np.full((n_bootstraps, n_periods), np.nan)

# Map & Lists (Run Once)
market_indices_map = df.groupby('market_id').indices
targeted_markets = df[df.target_routes == 1]['market_id'].unique()
non_targeted_markets = df[df.target_routes == 0]['market_id'].unique()

# --- SET SEED FOR REPRODUCIBILITY ---
# 42 is the answer to the ultimate question of life, the universe, and everything
np.random.seed(42) 

print(f"Starting {n_bootstraps} iterations...")

# --- 2. BOOTSTRAP LOOP ---
from tqdm import tqdm

for b in tqdm(range(n_bootstraps)):
    try:
        # A. Sample Markets (Cluster Resampling)
        samp_target_mkts = np.random.choice(targeted_markets, size=len(targeted_markets), replace=True)
        samp_nontarget_mkts = np.random.choice(non_targeted_markets, size=len(non_targeted_markets), replace=True)
        
        # B. Build Index Mask
        target_indices = [market_indices_map[m] for m in samp_target_mkts]
        nontarget_indices = [market_indices_map[m] for m in samp_nontarget_mkts]
        idx_mask = np.concatenate(target_indices + nontarget_indices)
        
        # C. Estimate
        base, mono, targ = estimate_coefficients_optimized(idx_mask)
        
        # D. Store
        base_bootstrap[b, :] = base.reindex(periods).values
        monopoly_bootstrap[b, :] = mono.reindex(periods).values
        targeted_bootstrap[b, :] = targ.reindex(periods).values

    except Exception as e:
        print(f"Error in bootstrap {b}: {e}")
        continue

print("Bootstrap complete!")

Starting 1000 iterations...


100%|██████████| 1000/1000 [25:07<00:00,  1.51s/it]

Bootstrap complete!


In [ ]:
# Calculate mean and confidence intervals (95%)
# We use 'nan*' functions to ignore missing periods without crashing
base_mean = np.nanmean(base_bootstrap, axis=0)
base_ci_lower = np.nanpercentile(base_bootstrap, 2.5, axis=0)
base_ci_upper = np.nanpercentile(base_bootstrap, 97.5, axis=0)

monopoly_mean = np.nanmean(monopoly_bootstrap, axis=0)
monopoly_ci_lower = np.nanpercentile(monopoly_bootstrap, 2.5, axis=0)
monopoly_ci_upper = np.nanpercentile(monopoly_bootstrap, 97.5, axis=0)

targeted_mean = np.nanmean(targeted_bootstrap, axis=0)
targeted_ci_lower = np.nanpercentile(targeted_bootstrap, 2.5, axis=0)
targeted_ci_upper = np.nanpercentile(targeted_bootstrap, 97.5, axis=0)

# Replace any remaining NaNs with zeros ONLY for plotting
# (e.g., if a period was missing in 100% of bootstraps)
base_mean = np.nan_to_num(base_mean)
base_ci_lower = np.nan_to_num(base_ci_lower)
base_ci_upper = np.nan_to_num(base_ci_upper)

monopoly_mean = np.nan_to_num(monopoly_mean)
monopoly_ci_lower = np.nan_to_num(monopoly_ci_lower)
monopoly_ci_upper = np.nan_to_num(monopoly_ci_upper)

targeted_mean = np.nan_to_num(targeted_mean)
targeted_ci_lower = np.nan_to_num(targeted_ci_lower)
targeted_ci_upper = np.nan_to_num(targeted_ci_upper)

In [ ]:
periods = list(range(-14,15))

fig = go.Figure()

# Base coefficient line and CI band
fig.add_trace(go.Scatter(
    x=periods,
    y=np.exp(base_coefs_main[periods]),
    mode='lines+markers',
    line=dict(color=color_palette[0], width=2),
    marker=dict(size=8, color=color_palette[0]),
    name='Not Intra-Hawaiian routes'
))

fig.add_trace(go.Scatter(
    x=periods + periods[::-1],
    y=np.exp(base_ci_upper[2:-2]).tolist() + np.exp(base_ci_lower[2:-2]).tolist()[::-1],
    fill='toself',
    fillcolor=color_palette[0],
    opacity=0.3,
    line=dict(color=color_palette[0]),
    hoverinfo="skip",
    showlegend=False
))

# Targeted coefficient line and CI band
fig.add_trace(go.Scatter(
    x=periods,
    y=np.exp(targeted_coefs_main[periods]),
    mode='lines+markers',
    line=dict(color=color_palette[1], width=2),
    marker=dict(size=8, color=color_palette[1]),
    name='Intra-Hawaiian routes'
))

fig.add_trace(go.Scatter(
    x=periods + periods[::-1],
    y=np.exp(targeted_ci_upper[2:-2]).tolist() + np.exp(targeted_ci_lower[2:-2]).tolist()[::-1],
    fill='toself',
    fillcolor=color_palette[1],
    opacity=0.3,
    line=dict(color=color_palette[1]),
    hoverinfo="skip",
    showlegend=False
))

# Monopoly coefficient line and CI band
fig.add_trace(go.Scatter(
    x=periods,
    y=np.exp(monopoly_coefs_main[periods]),
    mode='lines+markers',
    line=dict(color=color_palette[4], width=2),
    marker=dict(size=8, color=color_palette[4]),
    name='Markets that were monopoly before SW entry, excluding intra-Hawaiian'
))

fig.add_trace(go.Scatter(
    x=periods + periods[::-1],
    y=np.exp(monopoly_ci_upper[2:-2]).tolist() + np.exp(monopoly_ci_lower[2:-2]).tolist()[::-1],
    fill='toself',
    fillcolor=color_palette[4],
    opacity=0.3,
    line=dict(color=color_palette[4]),
    hoverinfo="skip",
    showlegend=False
))

# Reference lines
fig.add_shape(
    type="line",
    x0=min(periods), y0=1, x1=max(periods), y1=1,
    line=dict(color="black", width=2, dash="solid")
)

y_min = min(np.min(np.exp(base_ci_lower)), np.min(np.exp(targeted_ci_lower)), np.min(np.exp(monopoly_ci_lower)))
y_max = max(np.max(np.exp(base_ci_upper)), np.max(np.exp(targeted_ci_upper)), np.max(np.exp(monopoly_ci_upper)))

fig.add_shape(
    type="line",
    x0=-0.5, y0=y_min, x1=-0.5, y1=y_max,
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash="dash")
)

# Southwest entry annotation
fig.add_annotation(
    x=-0.5,
    y=y_max + 0.05,
    text="SW entry",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color='rgba(231, 17, 94, 1)'
    )
)

# Layout settings
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family="Arial",
        size=20
    ),
    xaxis=dict(
        title='Time Relative to Southwest Entry (Quarters)',
        title_font=dict(size=22),
        tickmode='array',
        tickvals=periods,
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title='Change in Average Market Price (Percent)',
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True,
        tickformat='.1%'
    ),
    legend=dict(
        font=dict(size=18),
        orientation="h",
        y=-0.15,
        x=0.5,
        xanchor="center",
        bordercolor="black",
        borderwidth=1
    ),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    margin=dict(t=50, b=100, l=100, r=40)
)

fig.show()

fig.write_image( PLOT_FOLDER/"main_plot.png", scale=3 )


# Robustness checks 

### Placebo test

In [ ]:
train_mask_placebo = (
    ((df.rel_period < -6) | (df.rel_period == -100)) & 
    ((df.hi_routes == 0) | (df.rel_period < -9))
).values
placebo_base_coefs_main, placebo_monopoly_coefs_main, placebo_targeted_coefs_main = estimate_coefficients_optimized(full_sample_mask, train_mask = train_mask_placebo)

Did visual inspection, since I dont have bootstrap for pplacebo, the plots look very clode to the original one for all 3 groups

### Pre-trend test

F-test on joint

In [ ]:
# Inputs
coefs = base_coefs_main  # pd.Series, index is relative period, values are point estimates
ci_lower = base_ci_lower  # np.ndarray
ci_upper = base_ci_upper  # np.ndarray

# Step 1: Extract pre-treatment periods (-16 to -7)
pre_periods = range(-14, -3)
pre_coefs = coefs.loc[pre_periods]

# Step 2: Estimate standard errors from confidence intervals
z_score = 1.96  # for 95% CI
std_errors = (ci_upper - ci_lower) / (2 * z_score)

# Create a Series for standard errors aligned with coefs
std_err_series = pd.Series(std_errors, index=coefs.index)
pre_se = std_err_series.loc[pre_periods]

# Step 3: Perform Wald test: H0 is that all pre_coefs == 0
# W = β'Σ⁻¹β  ~ χ²_k under H0
beta_vec = pre_coefs.values
cov_matrix = np.diag(pre_se.values**2)

# Compute Wald statistic
wald_stat = beta_vec @ np.linalg.inv(cov_matrix) @ beta_vec
df = len(pre_periods)
# Compute p-value from chi-squared distribution
p_value = 1 - chi2.cdf(wald_stat, df)

# Output results
print(f"Wald statistic: {wald_stat:.4f}")
print(f"Degrees of freedom: {df}")
print(f"P-value: {p_value:.10f}")


Wald statistic: 46.7322
Degrees of freedom: 11
P-value: 0.0000023991


z-test on average

In [ ]:
import numpy as np
from scipy.stats import norm

# inputs
alpha    = 0.05
z_crit   = norm.ppf(1 - alpha/2)   # ≈1.96
ses      = np.array(pre_se)
K        = len(ses)
se_avg   = np.sqrt((ses**2).sum()) / K
betas    = pre_coefs

beta_avg = betas.mean()
se_avg   = np.sqrt((ses**2).sum()) / K

# 5. z‐stat and p‐value
z_stat = beta_avg/ se_avg
p_val  = 2 * (1 - norm.cdf(abs(z_stat)))

print(f"Average pre‐trend β̄ = {beta_avg:.4f}")
print(f"SE(β̄) = {se_avg:.4f}")
print(f"z = {z_stat:.2f},  p = {p_val:.3f}")


Average pre‐trend β̄ = -0.0054
SE(β̄) = 0.0020
z = -2.69,  p = 0.007


t-test on average

In [ ]:

df       =  K-1  # e.g. K-1

# 2) specify your equivalence margin (±1 pp)
delta = 0.0201

# 3) compute test statistics
#   Test 1: H01: beta_avg <= -delta  vs.  > -delta
t1     = (beta_avg - (-delta)) / se_avg
p1_one = 1 - stats.t.cdf(t1, df)      # one-sided p-value for H01

#   Test 2: H02: beta_avg >=  delta  vs.  <  delta
t2     = (beta_avg -  delta) / se_avg
p2_one = stats.t.cdf(t2, df)          # one-sided p-value for H02

# 4) overall p-value for equivalence is the max of the two
p_equiv = max(p1_one, p2_one)

print(f"t1 = {t1:.2f}, p1 = {p1_one:.3f}")
print(f"t2 = {t2:.2f}, p2 = {p2_one:.3f}")
print(f"Equivalence test p = {p_equiv:.3f}")


t1 = 7.25, p1 = 0.000
t2 = -12.62, p2 = 0.000
Equivalence test p = 0.000


# Printing the results

In [ ]:
def create_bootstrap_latex_table(base_coefs, monopoly_coefs, targeted_coefs, 
                               base_bootstrap, monopoly_bootstrap, targeted_bootstrap,
                               targeted_periods=[-14, -6, -1, 0, 1, 6, 14],
                               confidence_level=0.95):
    """
    Create a LaTeX table from bootstrap results with 3 columns (base, monopoly, targeted)
    
    Parameters:
    ----------
    base_coefs : pandas Series
        Point estimates for base effect, indexed by period
    monopoly_coefs : pandas Series
        Point estimates for monopoly effect, indexed by period  
    targeted_coefs : pandas Series
        Point estimates for targeted effect, indexed by period
    base_bootstrap : pandas DataFrame or numpy array
        Bootstrap samples for base effect [bootstraps, periods]
    monopoly_bootstrap : pandas DataFrame or numpy array
        Bootstrap samples for monopoly effect [bootstraps, periods]
    targeted_bootstrap : pandas DataFrame or numpy array
        Bootstrap samples for targeted effect [bootstraps, periods]
    targeted_periods : list
        Periods to include in the table
    confidence_level : float
        Confidence level for intervals (default: 0.95)
    
    Returns:
    -------
    str
        LaTeX table string
    """
    # Print information about input arrays for debugging
    # print(f"Bootstrap array shapes: {base_bootstrap.shape}, {monopoly_bootstrap.shape}, {targeted_bootstrap.shape}")
    
    # Convert DataFrames to numpy arrays if necessary
    if isinstance(base_bootstrap, pd.DataFrame):
        base_bootstrap_array = base_bootstrap.values
    else:
        base_bootstrap_array = base_bootstrap
        
    if isinstance(monopoly_bootstrap, pd.DataFrame):
        monopoly_bootstrap_array = monopoly_bootstrap.values
    else:
        monopoly_bootstrap_array = monopoly_bootstrap
        
    if isinstance(targeted_bootstrap, pd.DataFrame):
        targeted_bootstrap_array = targeted_bootstrap.values
    else:
        targeted_bootstrap_array = targeted_bootstrap
    
    # Calculate alpha for confidence intervals
    alpha = 1 - confidence_level
    
    # Start the LaTeX table
    latex = "\\begin{table}[htbp]\n"
    latex += "    \\centering\n"
    latex += "    \\small\n"
    latex += "    \\caption{Effect of Southwest Entry on Prices}\n"
    latex += "    \\label{tab:sw_entry_effect}\n"
    latex += "    \\resizebox{\\textwidth}{!}{\n"
    latex += "        \\begin{tabular}{l>{\centering\\arraybackslash}p{2.5cm}>{\centering\\arraybackslash}p{2.5cm}>{\centering\\arraybackslash}p{2.5cm}}\n"
    latex += "            \\toprule\n"
    latex += "            & Base Effect & Monopoly Effect & Targeted Effect \\\\\n"
    latex += "            \\midrule\n"
    
    # Data rows for each period
    for period in targeted_periods:
        period_name = f"Period {period}" 
        period_float = float(period)  # Convert to float to match DataFrame index
        
        # Print period for debugging
        # print(f"Processing period {period_float}")
        
        # Check if period exists in the data
        if period_float not in base_coefs.index:
            print(f"Period {period_float} not found in coefficient data. Skipping.")
            continue
            
        # Find the position of this period in the bootstrap arrays
        # This assumes the bootstrap array columns correspond to the same ordering as the coefficients
        period_position = list(base_coefs.index).index(period_float)
        
        # Get point estimates directly from Series with float index
        base_coef = base_coefs.loc[period_float]
        monopoly_coef = monopoly_coefs.loc[period_float]
        targeted_coef = targeted_coefs.loc[period_float]
        
        # Get bootstrap samples for this period using the position
        if isinstance(base_bootstrap, pd.DataFrame):
            # If bootstrap is a DataFrame, use the column that corresponds to this period
            period_col = base_bootstrap.columns[period_position]
            base_boot_samples = base_bootstrap[period_col].values
            monopoly_boot_samples = monopoly_bootstrap[monopoly_bootstrap.columns[period_position]].values
            targeted_boot_samples = targeted_bootstrap[targeted_bootstrap.columns[period_position]].values
        else:
            # If bootstrap is already a numpy array, use column indexing
            base_boot_samples = base_bootstrap_array[:, period_position]
            monopoly_boot_samples = monopoly_bootstrap_array[:, period_position]
            targeted_boot_samples = targeted_bootstrap_array[:, period_position]
        
        # Calculate approximate p-values based on bootstrapped distribution
        base_p = 2 * min(np.mean(base_boot_samples <= 0), np.mean(base_boot_samples >= 0))
        monopoly_p = 2 * min(np.mean(monopoly_boot_samples <= 0), np.mean(monopoly_boot_samples >= 0))
        targeted_p = 2 * min(np.mean(targeted_boot_samples <= 0), np.mean(targeted_boot_samples >= 0))
        
        # Add stars based on p-values
        base_stars = ""
        if base_p < 0.01:
            base_stars = "$^{***}$"
        elif base_p < 0.05:
            base_stars = "$^{**}$"
        elif base_p < 0.1:
            base_stars = "$^{*}$"
        
        monopoly_stars = ""
        if monopoly_p < 0.01:
            monopoly_stars = "$^{***}$"
        elif monopoly_p < 0.05:
            monopoly_stars = "$^{**}$"
        elif monopoly_p < 0.1:
            monopoly_stars = "$^{*}$"
        
        targeted_stars = ""
        if targeted_p < 0.01:
            targeted_stars = "$^{***}$"
        elif targeted_p < 0.05:
            targeted_stars = "$^{**}$"
        elif targeted_p < 0.1:
            targeted_stars = "$^{*}$"
        
        # Format the coefficient values
        base_coef_str = f"{base_coef:.3f}".replace("0.", ".")
        if base_coef_str != ".000":
            base_coef_str += base_stars
        
        monopoly_coef_str = f"{monopoly_coef:.3f}".replace("0.", ".")
        if monopoly_coef_str != ".000":
            monopoly_coef_str += monopoly_stars
        
        targeted_coef_str = f"{targeted_coef:.3f}".replace("0.", ".")
        if targeted_coef_str != ".000":
            targeted_coef_str += targeted_stars
        
        # Coefficient row
        latex += f"            {period_name} & {base_coef_str} & {monopoly_coef_str} & {targeted_coef_str} \\\\\n"
        
        # Calculate standard errors from bootstrap
        base_se = np.std(base_boot_samples, ddof=1)
        monopoly_se = np.std(monopoly_boot_samples, ddof=1)
        targeted_se = np.std(targeted_boot_samples, ddof=1)
        
        # Format standard errors
        base_se_str = f"({base_se:.3f})".replace("0.", ".")
        monopoly_se_str = f"({monopoly_se:.3f})".replace("0.", ".")
        targeted_se_str = f"({targeted_se:.3f})".replace("0.", ".")
        
        # Standard error row
        latex += f"            & {base_se_str} & {monopoly_se_str} & {targeted_se_str} \\\\\n"
    
    # Add footer information
    latex += "            \\midrule\n"
    latex += "            Fixed Effects & \\multicolumn{3}{c}{Market, Time} \\\\\n"
    latex += "            COVID-19 Controls & \\multicolumn{3}{c}{Yes} \\\\\n"
    # latex += "            Observations & \\multicolumn{3}{c}{N} \\\\\n"  # Replace N with actual number
    latex += f"            Bootstrap Iterations & \\multicolumn{{3}}{{c}}{{{len(base_bootstrap)}}} \\\\\n"
    latex += "            \\bottomrule\n"
    latex += "        \\end{tabular}\n"
    latex += "    }\n"
    latex += "    \\vspace{0.5em}\n"
    latex += "    \\begin{minipage}{\\textwidth}\n"
    latex += "    \\scriptsize\n"
    latex += "    \\textit{Note:} Bootstrap standard errors in parentheses. $^{***}$ p$<$0.01, $^{**}$ p$<$0.05, $^{*}$ p$<$0.1. Period -1 is the reference period. Full period-by-period results are presented in the figures.\n"
    latex += "    \\end{minipage}\n"
    latex += "\\end{table}"
    
    return latex

In [ ]:
latex_table = create_bootstrap_latex_table(
    base_coefs_main,      # Series with point estimates for base effect
    monopoly_coefs_main,  # Series with point estimates for monopoly effect
    targeted_coefs_main,  # Series with point estimates for targeted effect
    base_bootstrap, # Bootstrap samples [bootstraps, periods]
    monopoly_bootstrap,
    targeted_bootstrap,
    targeted_periods=[-14, -6, -1, 0, 1, 6, 14]
)
print(latex_table)

\begin{table}[htbp]
    \centering
    \small
    \caption{Effect of Southwest Entry on Prices}
    \label{tab:sw_entry_effect}
    \resizebox{\textwidth}{!}{
        \begin{tabular}{l>{\centering\arraybackslash}p{2.5cm}>{\centering\arraybackslash}p{2.5cm}>{\centering\arraybackslash}p{2.5cm}}
            \toprule
            & Base Effect & Monopoly Effect & Targeted Effect \\
            \midrule
            Period -14 & .029$^{***}$ & -.016 & .026$^{*}$ \\
            & (.009) & (.018) & (.013) \\
            Period -6 & -.016$^{**}$ & .018 & .034$^{*}$ \\
            & (.007) & (.015) & (.018) \\
            Period -1 & -.024$^{**}$ & .046$^{***}$ & .074$^{***}$ \\
            & (.011) & (.015) & (.019) \\
            Period 0 & -.082$^{***}$ & -.063$^{***}$ & -.411$^{***}$ \\
            & (.011) & (.020) & (.037) \\
            Period 1 & -.152$^{***}$ & -.115$^{***}$ & -.514$^{***}$ \\
            & (.013) & (.023) & (.055) \\
            Period 6 & -.073$^{***}$ & -.071$^{***}$ 

# Load factor

In [2]:
load_factor_df = con.execute("""
                                    with 
                                          route_presence AS (
                                                            SELECT DISTINCT date, market_id, TkCarrier
                                                            FROM DB1B_processed
                                                            ),

                                          sw_entry AS (
                                                            SELECT market_id, MIN(date) AS SW_entry_date
                                                            FROM route_presence
                                                            WHERE TkCarrier='WN'
                                                            GROUP BY 1
                                                            having SW_entry_date >= '2017-01-01'
                                                            )
                                    select t1.date,
                                          t1.market_id,
                                          t1.TkCarrier,
                                          case when t1.market_id in (select distinct market_id from final_panel where target_routes = 1) then 1 else 0 end as target_routes,
                                          case when t1.market_id in (select distinct market_id from sw_entry) then 1 else 0 end as new_markets,
                                          t2.PASSENGERS,
                                          t2.SEATS,
                                          YEAR,
                                          QUARTER,
                                          DEPARTURES_PERFORMED
                                    FROM DB1B_processed as t1
                                    inner join valid_t100_keys_u as t2 ON t1.date = t2.date
                                                                        AND t1.market_id = t2.market_id
                                                                        AND t1.TkCarrier = t2.UNIQUE_CARRIER
                                    where TkCarrier in ['WN'] 
                                    and t1.date >= '2017-01-01'
                              """
                              ).df()

In [3]:
load_factor_df = load_factor_df.sort_values('date')
y1= load_factor_df.groupby('date')[['PASSENGERS', 'SEATS']].sum()
y1 = y1.PASSENGERS/y1.SEATS
y2= load_factor_df.query("new_markets == 1").groupby('date')[['PASSENGERS', 'SEATS']].sum()
y2 = y2.PASSENGERS/y2.SEATS
y3= load_factor_df.query("target_routes == 1").groupby('date')[['PASSENGERS', 'SEATS']].sum()
y3 = y3.PASSENGERS/y3.SEATS

In [4]:
# Create the figure
fig = go.Figure()

# Add traces for y1, y2, and y3
traces = [
    go.Scatter(
        x=y1.index, 
        y=y1.values, 
        mode='lines+markers',
        name='Systemwide load factor',
        line=dict(color=color_palette[0]),
        marker=dict(color=color_palette[0], symbol='circle', size=10)
    ),
    go.Scatter(
        x=y2.index, 
        y=y2.values, 
        mode='lines+markers',
        name='Markets Southwest entered since 2017',
        line=dict(color=color_palette[1]),
        marker=dict(color=color_palette[1], symbol='square', size=10)
    ),
    go.Scatter(
        x=y3.index, 
        y=y3.values, 
        mode='lines+markers',
        name='Intra-Hawaiian routes',
        line=dict(color=color_palette[4]),
        marker=dict(color=color_palette[4], symbol='triangle-up', size=10)
    )
]

# Add traces to the figure
for trace in traces:
    fig.add_trace(trace)

# Add COVID-19 shaded region
fig.add_shape(
    type="rect",
    x0="2020-02-01",
    x1="2021-04-01",
    y0=0,
    y1=1,
    fillcolor="lightgray",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# COVID annotation
fig.add_annotation(
    x='2020-08-10',
    y=0.9,
    text="COVID-19",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    ),
    align="center",
    bgcolor="white",
    opacity=0.7,
    borderpad=4,
)

# Layout adjustments
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family="Arial",
        size=20
    ),
    xaxis=dict(
        title="Date (Quarter)",
        title_font=dict(size=22),
        tickmode='auto',
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title='Avg Load Factor (Percent)',
        title_font=dict(size=22),
        tickfont=dict(size=18),
        tickformat='.1%',
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    legend=dict(
        font=dict(size=18),
        orientation="h",
        y=-0.15,
        x=0.5,
        xanchor="center",
        bordercolor="black",
        borderwidth=1
    ),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    margin=dict(t=50, b=100, l=100, r=40)
)

# Show the figure
fig.show()

fig.write_image( PLOT_FOLDER/"load_factors.png", scale=3 )

In [5]:
hi_mkt = load_factor_df.query("target_routes == 1")

def days_in_quarter(year, quarter):
    # Map each quarter to the three months it covers
    if quarter == 1:
        months = [1, 2, 3]
    elif quarter == 2:
        months = [4, 5, 6]
    elif quarter == 3:
        months = [7, 8, 9]
    else:
        months = [10, 11, 12]
    
    # Sum the number of days in each month for that quarter
    return sum(calendar.monthrange(year, m)[1] for m in months)

# Apply function row by row
hi_mkt['days_in_quarter'] = hi_mkt.apply(lambda row: days_in_quarter(row['Year'], row['Quarter']), axis=1)

hi_mkt['avg_flights_per_day'] = hi_mkt.DEPARTURES_PERFORMED/hi_mkt.days_in_quarter

hi_mkt = hi_mkt.groupby(['date'], as_index = False).avg_flights_per_day.sum()

fig = go.Figure()

# Add Southwest trace
fig.add_trace(   
    go.Scatter(
        x=hi_mkt.date, 
        y=hi_mkt.avg_flights_per_day, 
        mode='lines+markers',
        name='Southwest Airlines',
        line=dict(color=color_palette[0]),
        marker=dict(color=color_palette[0], symbol='circle', size=10)
    )
)

# COVID shaded period
fig.add_shape(
    type="rect",
    x0="2020-02-01",
    x1="2021-04-01",
    y0=hi_mkt.avg_flights_per_day.min(),
    y1=hi_mkt.avg_flights_per_day.max(),
    fillcolor="lightgray",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# COVID annotation
fig.add_annotation(
    x='2020-08-10',
    y=hi_mkt.avg_flights_per_day.max() - 10,
    text="COVID-19",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    ),
    align="center",
    bgcolor="white",
    opacity=0.7,
    borderpad=4,
)

# Layout styling
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family="Arial",
        size=20
    ),
    xaxis=dict(
        title="Date (Quarter)",
        title_font=dict(size=22),
        tickmode='auto',
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title='Avg # of Flights per Day',
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    legend=dict(
        font=dict(size=18),
        orientation="h",
        y=-0.15,
        x=0.5,
        xanchor="center",
        bordercolor="black",
        borderwidth=1
    ),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    margin=dict(t=50, b=100, l=100, r=40)
)

fig.show()

fig.write_image( PLOT_FOLDER/"capacity.png", scale=3 )
